In [1]:
import json
import geopandas as gpd
from pyinterpolate.core.pipelines.block_filter import smooth_blocks
from pyinterpolate.semivariogram.theoretical.theoretical import TheoreticalVariogram
from pyinterpolate.core.data_models.point_support import PointSupport, Blocks

In [2]:
vario = '/Users/szymonos/Documents/GitHub/biebrza-follow-up/notebooks/reg_2025_01_07'
tbd_f = '/Users/szymonos/Documents/GitHub/biebrza-follow-up/raw/poland/raw_lyme_disease/core_dataset.shp'
pop_f = 'ppoland.geojson'

In [3]:
# Load variogram

semi = TheoreticalVariogram()

with open(vario, 'r') as fin:
    params = json.load(fin)
    
params

{'name': 'spherical',
 'nugget': 0.0,
 'range': 88794.70257826854,
 'sill': 1722.230545739049,
 'direction': None}

In [4]:
# nugget: float
# sill: float
# rang: float
# variogram_model_type: str

In [5]:
params_2 = {
    'nugget': params['nugget'],
    'sill': params['sill'],
    'rang': params['range'],
    'variogram_model_type': params['name']
}

In [6]:
semi.from_dict(params_2)

In [7]:
tbd = gpd.read_file(tbd_f)
VAL_COL = '2015'
INDEX_COL = 'index1'
GEOM_COL = 'geometry'

In [8]:
pop = gpd.read_file(pop_f)
pop.head()

,h3_index_8,RES,geometry
0,881e018205fffff,0,POINT (560942.482 149294.741)
1,881e018207fffff,0,POINT (561836.233 149022.556)
2,881e01820bfffff,0,POINT (562373.316 150476.091)
3,881e01820dfffff,0,POINT (560764.22 150157.581)
4,881e018211fffff,0,POINT (564160.834 149931.799)


In [9]:
pop['RES'] = pop['RES'].astype(float)

In [10]:
POP_COL = 'RES'
POP_GEOM = 'geometry'

In [11]:
blocks = Blocks(ds=tbd, value_column_name=VAL_COL, geometry_column_name=GEOM_COL, index_column_name=INDEX_COL,
                representative_points_from_random_sample=True)

In [12]:
ps = PointSupport(
    points=pop,
    blocks=blocks,
    points_value_column=POP_COL,
    points_geometry_column=POP_GEOM,
    calculate_weighted_block_to_block_distances=True,
    calculate_distances_between_point_support_points=True,
    no_possible_neighbors=8,
    verbose=True
)

Calculating weighted distances between blocks...


100%|██████████| 328/328 [21:39<00:00,  3.96s/it]


Calculating distances between point supports...


100%|██████████| 328/328 [22:35<00:00,  4.13s/it]


In [15]:
smooth_blocks(
    semivariogram_model=semi,
    point_support=ps,
    number_of_neighbors=8,
    raise_when_negative_prediction=True,
    raise_when_negative_error=False,
    verbose=True
)

  0%|          | 0/328 [00:20<?, ?it/s]


ValueError: No objects to concatenate